# Copyright
<PRE>
Jelen iPython notebook a Budapesti Műszaki és Gazdaságtudományi Egyetemen tartott 
"Deep Learning a gyakorlatban Python és LUA alapon" tantárgy segédanyagaként készült, 
az alábbi forrás alapján: 
https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

A tantárgy honlapja: http://smartlab.tmit.bme.hu/oktatas-deep-learning 
Deep Learning kutatás: http://smartlab.tmit.bme.hu/deep-learning

A forráskódot GPLv3 licensz védi. Újrafelhasználás esetén lehetőség szerint kérjük az alábbi szerzőt értesíteni.

2016 (c) Szaszák György (szaszak kukac tmit pont bme pont hu)
</PRE>

## Dokumentumosztályozás

Szóbeágyazások felhasználásával a dokumentumok téma szerinti osztályozása a feladat. Ezútal GloVe beágyazásokat importálunk majd.

In [1]:
from __future__ import print_function
import os
import numpy as np
np.random.seed(1337)

# Importok, köztük egy text tokenizáló (szavakra bont, kidobálja a felesleges karaktereket)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
import sys

# GloVe beágyazásokat fogunk használni cikkek tematikus osztályozására
BASE_DIR = '.'
GLOVE_DIR = BASE_DIR + '/glove.6B/'
TEXT_DATA_DIR = BASE_DIR + '/20_newsgroup/'
MAX_NB_WORDS = 20000 # Ennyi különböző szót kezelünk majd
EMBEDDING_DIM = 100 # Ekkora lesz a használt beágyazás

Using TensorFlow backend.


## GloVe beágyazások importja
A beágyazások készen vannak egy fájlban, amelynek felépítése: szó, majd 100 elemű float vektor, amely a beágyazást adja meg. Ezeket beolvassuk egy asszociatív tömbbe.
Maga a beágyzás egy előre tanított súlymátrixból adódik, amelynek sorait címzik az egyes szavak.
A GloVe-t angol Wikipédián tanították elő. Elérhető:
http://nlp.stanford.edu/projects/glove/

In [2]:
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split() # "tokenizáljuk" a sort
    word = values[0] # maga a szó
    coefs = np.asarray(values[1:], dtype='float32') # a szót követő beágyazás 100 koordinátán
    embeddings_index[word] = coefs
f.close()

print('Betöltött beágyazások száma:', len(embeddings_index))
print('A `the` beágyazó vektorának első 10 eleme:', embeddings_index["the"][:10])

Betöltött beágyazásaok száma: 400000
A `the` beágyazó vektorának első 10 eleme: [-0.038194   -0.24487001  0.72812003 -0.39961001  0.083172    0.043953
 -0.39140999  0.3344     -0.57545     0.087459  ]


## Szövegkorpusz importja és előkészítése


In [6]:
texts = []  # lista a szövegekből vett sztring mintákkal.
labels_index = {}  # a topikokat egész azonosítóra képezzük (a topikokat a fájl neve adja meg)
labels = []  # topikok listáját közben kigyűjtjük
for name in sorted(os.listdir(TEXT_DATA_DIR)):
    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[name] = label_id # következő egész szám a következő ID
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                f = open(fpath, encoding='latin-1')
                texts.append(f.read())
                f.close()
                labels.append(label_id)

print('Összes szöveg: ', len(texts))
#print('Összes címke: ', len(labels))

Összes szöveg:  19997
Összes címke:  19997


## A szövegminták vektorizálása
Ehhez szavakra kell bontanunk a szövegeket, a szövegeket vágni valamilyen maximális hosszra, illetve a rövidebbeket kiegészíteni ilyen hosszra, hogy a bemeneti szekvencia mindig azonos hosszúságú (itt most 1000) legyen.

In [8]:
# Tokenizálás
tokenizer = Tokenizer(num_words=MAX_NB_WORDS) # Tokenizál, legfeljebb MAX_NB_WORDS szóra; token=szó
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts) # Soronként egy-egy szöveg szavai következnek egymás után listában.

word_index = tokenizer.word_index
print('Különböző szavak száma az összes szövegben: ', len(word_index))

# Vágjuk / kiegészítjük mindet 1000 szó hosszúra
data = pad_sequences(sequences, maxlen=1000)

labels = to_categorical(np.asarray(labels))
print('A data tenzor alakja:', data.shape)
print('A label tenzor alakja:', labels.shape)

# Csinálunk egy validációs halmazt.
# Teszthalmazunk most nem lesz, a tanítás során a validációs halmazra kapunk majd accuracy értéket,
# ezt éles alkalmazásban ne csináljuk!
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(0.2 * data.shape[0])

# Keverés után marad egy tanító és egy validációs halmaz
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

Különböző szavak száma az összes szövegben:  214873
A label tenzor alakja: 19997
A data tenzor alakja: (19997, 1000)
A label tenzor alakja: (19997, 20)


## Beágyazás alkalmazása a szövegekre
Arra is figyelnünk kell, ha olyan szót találunk, amelynek nincs beágyazása (kinullázzuk)

In [9]:
nb_words = min(MAX_NB_WORDS, len(word_index)) # Nincs értelme nagyobbra lőni, mint az adott szótárméret a GloVe-ban
embedding_matrix = np.zeros((nb_words + 1, EMBEDDING_DIM)) # Beágyazási mátrix, az importáltból fogjuk a sajátjunkat szemezgetni és átindexelni
for word, i in word_index.items():
    if i > MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: # különben csupa nulla lesz a beágyazóvektor
        embedding_matrix[i] = embedding_vector
        
print ('A szövegre alkalmazott beágyazási mátrix sorainak száma:', len(embedding_matrix))

A szövegre alkalmazott beágyazási mátrix sorainak száma: 20001


## Beágyazó réteg elkészítése
Csinálunk egy beágyazó réteget, az lesz a háló elején. Súlyait befagyasztjuk, 

In [10]:
embedding_layer = Embedding(nb_words + 1,
                            EMBEDDING_DIM, # Ilyen hosszúak a beágyazó vektoraink
                            weights=[embedding_matrix],
                            input_length=1000, # Ez az első 1000 szó minden szövegrészből
                            trainable=False) # Súlyokat nem hagyjuk módosítani


# Tanítunk egy 1d konvolúciós hálót osztályozásra
sequence_input = Input(shape=(1000,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences) # 128 5-ös szűrő fut végig a szövegeken háromszor
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(35)(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=2, batch_size=128)


/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:27: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 252s 16ms/step - loss: 1.4853 - acc: 0.5210 - val_loss: 0.2310 - val_acc: 0.9380
Epoch 2/2
15998/15998 [==============================] - 253s 16ms/step - loss: 0.2018 - acc: 0.9353 - val_loss: 0.1361 - val_acc: 0.9470
